### 階層構造

#### *_test.f90
テスト用ファイル
CGはshiftのない単なる行列方程式を解くためのコード
COCGはshiftのある場合のコード

↓

#### COCG_kernel.f90
COCGに方程式を渡して解を出力するまでの一連の流れを書いたファイル
現在はCG用とCOCG用の2つ。実際にHFBを解く際にはHermite行列用のコードを作る必要がある
後述するAp, rrはここで割り当てている。メモリのガベージを減らすため。

構造自体はシンプルで、指定の精度が得られるまでCOCGまたはCGのステップを繰り返すというもの。
収束のチェックには、残差rのsigmaに対する総和を計算している（CGの場合は単に残差の絶対値和）。

変数は
matrix : H(下階層では論文に合わせてAと表記。統一した方がいいかも)
vector : r, x, p 
scalar : alpha, beta, pi

添え字の表記は統一してあり、例えばr_nやx_nはrn, xn、r_{n-1}はrn_1などと表記する。
添え字自体はつなげて書き、マイナスをアンダーバーで置き換えるイメージ。

上付きsigmaの文字は末尾にsをつけることで表記。
ただしxns, rnsなどにはもとのxn, rnの情報も含んでいる（変数を一括して扱いたいため）。
xns, rnsの配列サイズは(N, 0:Nsigma)。sigma = 0の列がxn, rnに相当する。
この辺は別の配列で扱ってもいい気はしているが考え中。

↓

#### COCG_procedures.f90
COCGのステップ内部を記述している。
実際の計算はCOCG_componentに丸投げでルーチンを呼び出すだけ。

Ap, rrはそれぞれAp_{n-1}とr_{n-1} * r_{n-1}。
これらは各ステップで値を2回使うことになるが、各ルーチンで計算しなおしてると時間がかかってしまう（特に行列とベクトルの積）
そこで先に計算して使いまわすことで計算時間の短縮を図っている。
え、procedure内部で計算式全部ベタ書きすればいいだろって？黙れ。

手続きは通常のCGと、sigmaに対するshiftを計算するものの2つ。
sigmaは一度に計算するのではなく一つのsigmaに対する計算を記述している。

↓

#### COCG_components.f90
実際の計算式を格納している。
変数の順番はINTENTがin, outの順。同じグループではvector→scalarの順
Calc_Next_**がステップ計算。Calc_Shifted_はsigmaつき変数の計算

### 計算手順
行列のソースはJuliaを用いて100×100の乱数で生成。
CGは対称行列でないと使えないため、生成後対称化する（H = ( H^T + H ) / 2）。
生成した行列の各要素をHseedに格納。プログラム実行時に読み出す。

sigmaは0.1ごとに1までの10点を取り計算した。

解xを「CGtest_output」ファイルに出力し、逆行列を計算して解いた厳密解（Juliaで計算）と値を比較した。

### 計算結果
CGの計算結果。まずは厳密解。
